# Sales Forecasting

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import datetime
from pathlib import Path

/home/saketh/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [60]:
class LSTM_Final(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM_Final, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [65]:
def forecast_data(model,store):
    if model == "salesAnalyzer":
        PATH = "./predictions/salesAnalyzer/forecast/store_{}.csv".format(store)
        if not Path(PATH).is_file():
            print("Run Sales Analyzer Main File")
            
        forecast = pd.read_csv(PATH)
    elif model == "weatherAnalyzer":
        PATH = "./predictions/weatherAnalyzer/forecast/store_{}.csv".format(store)
        if not Path(PATH).is_file():
            print("Run Weather Analyzer Main File")
            
        forecast = pd.read_csv(PATH)
    elif model == "IBMCloud":
        PATH = "./IBM_Cloud/predictions/model-1/predictions.csv".format(store)
        if not Path(PATH).is_file():
            print("Run Weather Analyzer Main File")
            
        forecast = pd.read_csv(PATH)
        forecast = forecast[forecast["SCRUB_STORE_NO"] == store]
    else:
        forecast = None
    
    return forecast

In [66]:
def getScalingValues():
    try:
        model2_total = pd.read_csv("./predictions/salesAnalyzer/total/store_{}.csv".format(store))
        model2_total = model2_total.iloc[:-16,:]

        model3_total = pd.read_csv("./IBM_Cloud/predictions/model-1/predictions_total.csv".format(store))
        model3_total = model3_total[model3_total["SCRUB_STORE_NO"] == store].iloc[14:,:]

        training = pd.read_csv("./LSTM - Weather Sales Forecast/datasets/training.csv")
        training = training[training["SCRUB_STORE_NO"] == store].iloc[14:,:]["SALES_UNITS"].to_numpy()

        dataset = model2_total.iloc[:,-2:].copy()
        b = model3_total.iloc[:,-1:].copy()
        dataset["FORECAST_3"] = b["predictions"].to_numpy()
        dataset["ORIGINAL"] = training
    except:
        model2_total = pd.read_csv("./predictions/salesAnalyzer/total/store_{}.csv".format(store))
        model2_total = model2_total.iloc[:-15:]

        model3_total = pd.read_csv("./IBM_Cloud/predictions/model-1/predictions_total.csv".format(store))
        model3_total = model3_total[model3_total["SCRUB_STORE_NO"] == store].iloc[14:,:]

        training = pd.read_csv("./LSTM - Weather Sales Forecast/datasets/training.csv")
        training = training[training["SCRUB_STORE_NO"] == store].iloc[14:,:]["SALES_UNITS"].to_numpy()

        dataset = model2_total.iloc[:,-2:].copy()
        b = model3_total.iloc[:,-1:].copy()
        dataset["FORECAST_3"] = b["predictions"].to_numpy()
        dataset["ORIGINAL"] = training
    
    y = dataset.iloc[:,3:].values
    
    return y

In [68]:
file_data = pd.ExcelFile("../AI-ML Tractor Analytics/DataSet.xlsx")
forecast_excel = pd.read_excel(file_data,"FORECAST_DAY")

result = None

for store in range(1,21):
    model1_forecast = forecast_data("weatherAnalyzer",store)
    model2_forecast = forecast_data("salesAnalyzer",store)
    model3_forecast = forecast_data("IBMCloud",store)

    dataset = model1_forecast.iloc[:,-1:].copy()
    dataset["FORECAST_2"] = model1_forecast.iloc[:,-1:].to_numpy()
    dataset["FORECAST_3"] = model3_forecast.iloc[:,-1:].to_numpy()

    PATH = "./saved-models/saved_model_store_{}.pth".format(store)

    input_size = 3
    hidden_size = 40
    num_layers = 1 
    num_classes = 1 

    lstm = LSTM_Final(num_classes, input_size, hidden_size, num_layers)
    lstm.load_state_dict(torch.load(PATH))
    lstm.eval()

    X = dataset.values
    y = getScalingValues()

    ss = StandardScaler()
    mm = MinMaxScaler()

    X_ss = ss.fit_transform(X)
    y_mm = mm.fit_transform(y)

    data_X = Variable(torch.Tensor(X_ss))
    X_test_tensors = Variable(torch.Tensor(data_X))
    X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) 
    X_val = X_test_tensors_final

    train_predict = lstm(X_val)

    data_predict = train_predict.data.numpy()
    data_predict = mm.inverse_transform(data_predict).astype(int)
    

    forecast_day = forecast_excel[forecast_excel["SCRUB_STORE_NO"] == store].copy()
    forecast_day["FORECAST"] = data_predict
    forecast_day.to_csv("./predictions/main/store_{}.csv".format(store))
    

In [69]:
forecast_day

,SCRUB_STORE_NO,DATE_VALID_STD,DOY_STD,MIN_TEMPERATURE_AIR_2M_F,AVG_TEMPERATURE_AIR_2M_F,MAX_TEMPERATURE_AIR_2M_F,MIN_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_WETBULB_2M_F,MAX_TEMPERATURE_WETBULB_2M_F,MIN_TEMPERATURE_DEWPOINT_2M_F,...,MIN_CLOUD_COVER_TOT_PCT,AVG_CLOUD_COVER_TOT_PCT,MAX_CLOUD_COVER_TOT_PCT,MIN_RADIATION_SOLAR_TOTAL_WPM2,AVG_RADIATION_SOLAR_TOTAL_WPM2,MAX_RADIATION_SOLAR_TOTAL_WPM2,TOT_RADIATION_SOLAR_TOTAL_WPM2,PROBABILITY_OF_PRECIPITATION_PCT,PROBABILITY_OF_SNOW_PCT,FORECAST
301,20,2021-09-08,251,60.5,69.3,79.3,59.4,66.2,72.3,58.4,...,0,29,100,0,240.4,777.0,5528.1,100,0,225
302,20,2021-09-09,252,57.5,65.3,72.9,55.3,62.1,65.5,53.8,...,0,88,100,0,112.7,621.4,2706.1,95,0,160
303,20,2021-09-10,253,52.8,60.8,71.7,51.5,55.7,61.5,48.6,...,0,15,100,0,228.0,805.4,5471.4,3,0,118
304,20,2021-09-11,254,51.9,62.2,72.7,49.6,57.5,64.9,47.6,...,0,9,95,0,246.8,794.1,5923.7,0,0,135
305,20,2021-09-12,255,58.3,68.3,78.2,57.3,64.1,70.5,56.5,...,6,43,100,0,218.8,774.4,5250.1,0,0,78
306,20,2021-09-13,256,63.6,72.4,82.0,62.2,67.9,73.6,61.1,...,7,28,57,0,231.8,734.9,5562.9,25,0,-57
307,20,2021-09-14,257,66.4,73.3,83.9,64.7,68.9,74.8,63.3,...,24,67,100,0,201.8,615.2,4843.7,30,0,140
308,20,2021-09-15,258,59.6,64.3,71.6,55.4,61.0,65.5,51.6,...,98,100,100,0,62.5,205.2,1500.5,31,0,131
309,20,2021-09-16,259,53.6,61.7,70.7,50.6,55.2,60.4,47.2,...,0,59,100,0,179.1,579.0,4298.4,40,0,288
310,20,2021-09-17,260,51.1,61.3,72.8,48.6,55.2,62.3,45.4,...,0,0,0,0,238.2,759.7,5716.5,5,0,76


In [57]:
forecast_day

,SCRUB_STORE_NO,DATE_VALID_STD,DOY_STD,MIN_TEMPERATURE_AIR_2M_F,AVG_TEMPERATURE_AIR_2M_F,MAX_TEMPERATURE_AIR_2M_F,MIN_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_WETBULB_2M_F,MAX_TEMPERATURE_WETBULB_2M_F,MIN_TEMPERATURE_DEWPOINT_2M_F,...,TOT_SNOWFALL_IN,MIN_CLOUD_COVER_TOT_PCT,AVG_CLOUD_COVER_TOT_PCT,MAX_CLOUD_COVER_TOT_PCT,MIN_RADIATION_SOLAR_TOTAL_WPM2,AVG_RADIATION_SOLAR_TOTAL_WPM2,MAX_RADIATION_SOLAR_TOTAL_WPM2,TOT_RADIATION_SOLAR_TOTAL_WPM2,PROBABILITY_OF_PRECIPITATION_PCT,PROBABILITY_OF_SNOW_PCT
0,1,2021-09-08,251,52.1,60.9,69.2,50.6,55.7,58.8,49.3,...,0,0,25,100,0,206.8,711.5,4755.7,15,0
1,1,2021-09-09,252,51.0,58.7,70.3,49.7,53.5,58.5,47.3,...,0,0,45,100,0,204.1,727.4,4899.1,30,0
2,1,2021-09-10,253,49.6,61.2,73.8,48.2,54.1,60.2,46.1,...,0,0,27,100,0,228.8,769.3,5491.4,0,0
3,1,2021-09-11,254,57.2,68.2,79.2,54.5,61.8,65.9,50.8,...,0,7,67,100,0,180.8,704.1,4339.6,5,0
4,1,2021-09-12,255,55.3,65.5,77.2,53.4,57.0,62.6,43.7,...,0,5,74,100,0,185.6,723.4,4453.4,13,0
5,1,2021-09-13,256,56.9,67.2,81.4,52.1,61.7,69.8,45.6,...,0,13,66,100,0,138.1,519.6,3316.0,20,0
6,1,2021-09-14,257,52.3,63.1,71.8,51.1,58.2,62.0,47.4,...,0,0,46,100,0,226.0,708.2,5425.0,30,0
7,1,2021-09-15,258,50.1,59.8,71.6,48.7,53.3,58.2,45.5,...,0,0,42,100,0,159.0,487.3,3814.3,25,0
8,1,2021-09-16,259,51.0,62.0,77.3,48.0,54.0,60.5,44.5,...,0,0,0,1,0,230.8,699.5,5539.0,15,0
9,1,2021-09-17,260,54.0,64.3,78.2,48.5,55.9,62.2,42.5,...,0,0,0,0,0,227.4,690.2,5455.7,15,0
